---
execute:
  eval: true
  echo: true
  output: true
title: "web scraping"
---

- code examples [beatifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/){target="_blank"}
- code examples [scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html){target="_blank"}
- code examples [playwright](https://playwright.dev/python/docs/intro){target="_blank"}
- download [jupyter notebook](pyws05-1-data-collection.ipynb){target="_blank"}

In [ ]:
# run inside google colab
#!git clone https://github.com/cca-cce/osm-cdc.git

- example [sustainability communication](https://www.lunduniversity.lu.se/about-university/university-glance/mission-vision-and-values/sustainability){target="_blank"}

## ethical scraping, robots.txt

In [ ]:
import requests
import pandas as pd
from urllib.parse import urlparse, urljoin
import re

# Function to fetch and parse robots.txt into a DataFrame
def parse_robots_txt(robots_url):
    response = requests.get(robots_url)
    if response.status_code != 200:
        print(f"Error fetching robots.txt from {robots_url}")
        return None
    lines = response.text.splitlines()
    data = []
    current_user_agent = None
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#'):
            continue  # Skip empty lines and comments
        if line.lower().startswith('user-agent:'):
            current_user_agent = line.split(':', 1)[1].strip()
        elif line.lower().startswith(('disallow:', 'allow:')):
            directive, path = line.split(':', 1)
            directive = directive.strip()
            path = path.strip()
            data.append({
                'User-agent': current_user_agent,
                'Directive': directive.capitalize(),
                'Path': path
            })
    df = pd.DataFrame(data)
    return df

# Function to check if a URL is allowed or disallowed
def is_url_allowed(url, df, user_agent='*'):
    parsed_url = urlparse(url)
    path = parsed_url.path
    # Filter rules for the specific user-agent or '*'
    user_agent_rules = df[df['User-agent'].isin([user_agent, '*'])]

    # Initialize default permission
    permission = 'Allow'

    # Process rules in order
    for _, row in user_agent_rules.iterrows():
        rule_path = row['Path']
        directive = row['Directive']
        # Convert wildcard patterns to regular expressions
        rule_regex = '^' + re.escape(rule_path).replace(r'\*', '.*') + '$'
        if re.match(rule_regex, path):
            permission = directive
    return permission == 'Allow'

# Example usage
if __name__ == "__main__":
    # Define the URL of the robots.txt file
    base_url = 'https://www.example.com'  # Replace with the base URL
    base_url = 'https://www.lunduniversity.lu.se'  # Replace with the base URL
    robots_url = urljoin(base_url, '/robots.txt')

    # Parse the robots.txt file into a DataFrame
    robots_df = parse_robots_txt(robots_url)

    if robots_df is not None:
        print("Robots.txt rules:")
        print(robots_df)

        # URL to check
        url_to_check = 'https://www.example.com/some/path'  # Replace with the URL you want to check
        url_to_check = 'https://www.lunduniversity.lu.se/about-university/university-glance/mission-vision-and-values/sustainability'  # Replace with the URL you want to check

        # Check if the URL is allowed
        is_allowed = is_url_allowed(url_to_check, robots_df)

        if is_allowed:
            print(f"The URL {url_to_check} is allowed to be scraped.")
        else:
            print(f"The URL {url_to_check} is disallowed to be scraped according to robots.txt.")

## beautifulsoup, google colab

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import re

# Step 1: Define the URL to scrape
url = 'https://www.example.com'  # Replace with the target URL

# Step 2: Fetch the HTML content
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    exit()

html_content = response.text

# Step 3: Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Extract all link URLs to a DataFrame called links_df
links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    full_url = urljoin(url, href)
    links.append(full_url)

links_df = pd.DataFrame({'url': links})

# Step 5: Extract all image URLs to a DataFrame called images_df
images = []
for img in soup.find_all('img', src=True):
    src = img['src']
    full_url = urljoin(url, src)
    images.append(full_url)

images_df = pd.DataFrame({'image_url': images})

# Step 6: Extract the main text as markdown to texts_df
# We'll try to exclude navigation, footer, and other UI elements

# Attempt to find the main content
main_content = soup.find('main')
if not main_content:
    # If there's no <main> tag, find the largest <div> or <article>
    candidates = soup.find_all(['div', 'article'], recursive=True)
    main_content = max(candidates, key=lambda el: len(el.get_text(strip=True)), default=None)

if main_content:
    # Remove script and style elements
    for script_or_style in main_content(['script', 'style']):
        script_or_style.decompose()

    # Get text
    text_content = main_content.get_text(separator='\n', strip=True)

    # Optionally, clean up the text
    text_content = re.sub('\n+', '\n', text_content).strip()
else:
    text_content = ''

texts_df = pd.DataFrame({'text': [text_content]})

# Step 7: Save all DataFrames as TSV files
links_df.to_csv('links.tsv', sep='\t', index=False)
images_df.to_csv('images.tsv', sep='\t', index=False)
texts_df.to_csv('texts.tsv', sep='\t', index=False)

print("DataFrames have been saved as TSV files.")

---

### **Explanation:**

**Step 1: Define the URL to Scrape**

We start by specifying the URL of the page we want to scrape.

In [ ]:
url = 'https://www.example.com'  # Replace with the target URL

**Step 2: Fetch the HTML Content**

We use the `requests` library to send an HTTP GET request to the URL. If the request is successful (status code 200), we proceed; otherwise, we exit the script.

In [ ]:
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    exit()

html_content = response.text

**Step 3: Parse the HTML Content with BeautifulSoup**

We parse the HTML content using BeautifulSoup, which allows us to navigate and extract data from the HTML.

In [ ]:
soup = BeautifulSoup(html_content, 'html.parser')

**Step 4: Extract All Link URLs to a DataFrame Called `links_df`**

We find all `<a>` tags with an `href` attribute, resolve relative URLs to absolute URLs using `urljoin`, and store them in a list. We then create a DataFrame from this list.

In [ ]:
links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    full_url = urljoin(url, href)
    links.append(full_url)

links_df = pd.DataFrame({'url': links})

**Step 5: Extract All Image URLs to a DataFrame Called `images_df`**

Similarly, we find all `<img>` tags with a `src` attribute, resolve URLs, and store them in a DataFrame.

In [ ]:
images = []
for img in soup.find_all('img', src=True):
    src = img['src']
    full_url = urljoin(url, src)
    images.append(full_url)

images_df = pd.DataFrame({'image_url': images})

**Step 6: Extract the Main Text as Markdown to `texts_df`**

Extracting the main content while excluding navigation, footer, and other UI elements can be challenging. We attempt the following:

- **Check for a `<main>` Tag:** Many modern websites use the `<main>` tag to wrap the main content.
- **Fallback to the Largest `<div>` or `<article>` Tag:** If no `<main>` tag is found, we select the `<div>` or `<article>` with the most text content.
- **Clean the Text:** We remove any `<script>` or `<style>` tags from the main content and extract the text.

In [ ]:
# Attempt to find the main content
main_content = soup.find('main')
if not main_content:
    # If there's no <main> tag, find the largest <div> or <article>
    candidates = soup.find_all(['div', 'article'], recursive=True)
    main_content = max(candidates, key=lambda el: len(el.get_text(strip=True)), default=None)

if main_content:
    # Remove script and style elements
    for script_or_style in main_content(['script', 'style']):
        script_or_style.decompose()

    # Get text
    text_content = main_content.get_text(separator='\n', strip=True)

    # Optionally, clean up the text
    text_content = re.sub('\n+', '\n', text_content).strip()
else:
    text_content = ''

texts_df = pd.DataFrame({'text': [text_content]})

**Note:** This method provides an approximation. For more accurate extraction, consider using libraries like **Readability** or **Boilerpipe** that are designed to extract the main content from web pages.

**Step 7: Save All DataFrames as TSV Files**

We save each DataFrame to a TSV (Tab-Separated Values) file using pandas' `to_csv` method with `sep='\t'`.

In [ ]:
links_df.to_csv('links.tsv', sep='\t', index=False)
images_df.to_csv('images.tsv', sep='\t', index=False)
texts_df.to_csv('texts.tsv', sep='\t', index=False)

print("DataFrames have been saved as TSV files.")

---

### **Additional Notes:**

- **Dependencies:**
  - Ensure you have the required libraries installed:

    ```bash
    pip install requests beautifulsoup4 pandas
    ```

- **URL Replacement:**
  - Replace `'https://www.example.com'` with the actual URL you wish to scrape.

- **Handling Relative URLs:**
  - The `urljoin` function from `urllib.parse` resolves relative URLs based on the base URL.

- **Data Cleaning:**
  - The code removes extra newline characters in the extracted text.
  - For more advanced text cleaning and markdown conversion, consider using additional libraries like `markdownify`.

- **Legal and Ethical Considerations:**
  - **Respect Robots.txt:** Ensure that you are allowed to scrape the website by checking its `robots.txt` file.
  - **Terms of Service:** Review the website's terms of service to make sure scraping is permitted.
  - **Rate Limiting:** Be considerate and do not overwhelm the server with too many requests in a short period.

- **Error Handling:**
  - The script checks if the page was successfully retrieved before proceeding.
  - You can add more error handling as needed.

---

### **Example Usage:**

Suppose we set:

In [ ]:
url = 'https://www.python.org/'

After running the script, you would get:

- **links.tsv:** A TSV file containing all the links found on the Python homepage.
- **images.tsv:** A TSV file containing all the image URLs.
- **texts.tsv:** A TSV file containing the main textual content of the page.

---

**Feel free to customize the script further to suit your specific needs!**

## scrapy, github codespaces

In [ ]:
# scrapy_spider.py

import scrapy
from scrapy.crawler import CrawlerProcess
from urllib.parse import urljoin
import pandas as pd
import re

class MySpider(scrapy.Spider):
    name = "my_spider"

    # Start URL
    start_urls = ['https://www.example.com']  # Replace with your target URL

    # Initialize data storage
    links = []
    resources = []
    texts = []

    def parse(self, response):
        # Extract all link URLs
        link_urls = response.css('a::attr(href)').getall()
        for href in link_urls:
            full_url = urljoin(response.url, href)
            self.links.append(full_url)

        # Extract all binary file URLs (images, .pdf, .docx)
        # Define patterns for binary files
        binary_patterns = re.compile(r'.*\.(jpg|jpeg|png|gif|bmp|tiff|pdf|docx|doc)$', re.IGNORECASE)
        resource_urls = []

        # Extract image URLs
        img_urls = response.css('img::attr(src)').getall()
        resource_urls.extend(img_urls)

        # Include URLs from link tags that point to binary files
        for href in link_urls:
            if binary_patterns.match(href):
                resource_urls.append(href)

        # Resolve and store resource URLs
        for res_url in resource_urls:
            full_res_url = urljoin(response.url, res_url)
            if binary_patterns.match(full_res_url):
                self.resources.append(full_res_url)

        # Extract main text
        main_text = self.extract_main_text(response)
        self.texts.append(main_text)

    def extract_main_text(self, response):
        # Try to find <main> tag content
        main_content = response.css('main')
        if not main_content:
            # Fallback to the largest <div> or <article> tag
            candidates = response.css('div, article')
            if candidates:
                main_content = max(candidates, key=lambda el: len(el.get()), default=None)
            else:
                main_content = None
        else:
            main_content = main_content[0]

        if main_content:
            # Remove script and style tags
            main_content = main_content.xpath('.//*[not(self::script or self::style)]')
            text = main_content.xpath('.//text()').getall()
            text = [t.strip() for t in text if t.strip()]
            text_content = '\n'.join(text)
        else:
            text_content = ''

        return text_content

    def closed(self, reason):
        # Save data to dataframes
        links_df = pd.DataFrame({'url': self.links})
        resources_df = pd.DataFrame({'resource_url': self.resources})
        texts_df = pd.DataFrame({'text': self.texts})

        # Save dataframes to TSV files
        links_df.to_csv('links.tsv', sep='\t', index=False)
        resources_df.to_csv('resources.tsv', sep='\t', index=False)
        texts_df.to_csv('texts.tsv', sep='\t', index=False)

        print("DataFrames have been saved as TSV files.")

# Run the spider
if __name__ == "__main__":
    process = CrawlerProcess({
        'ROBOTSTXT_OBEY': True,
    })
    process.crawl(MySpider)
    process.start()

### Explanation:

**Imports and Setup:**

- **scrapy**: The main library for web scraping.
- **CrawlerProcess**: To run the spider without using the command line.
- **urljoin**: To resolve relative URLs.
- **pandas**: To create and manipulate DataFrames.
- **re**: For regular expressions.

**Define the Spider Class:**

- **name**: Name of the spider.
- **start_urls**: List containing the starting URL(s) for the spider.
- **Data Storage**: Initialize lists to store links, resources, and texts.

**Parsing the Response:**

- **Extract All Link URLs:**

  - Use CSS selectors to get all `<a>` tags with `href` attributes.
  - Resolve relative URLs using `urljoin`.
  - Store the full URLs in the `links` list.

- **Extract All Binary File URLs:**

  - Define a regular expression pattern to match image files and documents.
  - Extract `src` attributes from `<img>` tags.
  - Include URLs from `<a>` tags that point directly to binary files.
  - Resolve and store the full resource URLs in the `resources` list.

- **Extract Main Text:**

  - Use the `extract_main_text` method.
  - Attempt to extract content from the `<main>` tag.
  - If `<main>` tag is not found, fallback to the largest `<div>` or `<article>` tag.
  - Remove `<script>` and `<style>` tags.
  - Extract text content.
  - Store the text in the `texts` list.

**Closing the Spider:**

- When the spider finishes, the `closed` method is called.
- Convert the lists to pandas DataFrames.
- Save each DataFrame to a TSV file.

**Running the Spider:**

- Use `CrawlerProcess` to run the spider programmatically.
- Set `ROBOTSTXT_OBEY` to `True` to respect `robots.txt`.
- Call `process.start()` to begin crawling.

### How to Use:

1. **Install Scrapy and Pandas:**

   ```bash
   pip install scrapy pandas
   ```

2. **Save the Script:**

   - Save the code above in a file named `scrapy_spider.py`.

3. **Replace the URL:**

   - In the `start_urls` list, replace `'https://www.example.com'` with the URL you wish to scrape.

4. **Run the Script:**

   ```bash
   python scrapy_spider.py
   ```

5. **Check the Output:**

   - After the spider finishes, you will find three TSV files in the same directory:
     - `links.tsv`
     - `resources.tsv`
     - `texts.tsv`

### Additional Notes:

- **Legal and Ethical Considerations:**

  - **Respect Robots.txt:** The code sets `ROBOTSTXT_OBEY` to `True` to ensure compliance with the website's `robots.txt` policies.

  - **Terms of Service:** Before scraping, check the website's terms of service to ensure that scraping is permitted.

  - **Rate Limiting:** Be considerate and do not overload the server with too many requests. You can set a download delay:

In [ ]:
process = CrawlerProcess({
    'ROBOTSTXT_OBEY': True,
    'DOWNLOAD_DELAY': 1,  # Delay of 1 second between requests
})

- **Scrapy Settings:**

  - You can customize the Scrapy settings by passing a dictionary to `CrawlerProcess`.

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'my-scraper (https://www.mywebsite.com)',
    'ROBOTSTXT_OBEY': True,
    'DOWNLOAD_DELAY': 1,
    'LOG_LEVEL': 'INFO',  # Set to 'DEBUG' for more details
})

- **Handling JavaScript-rendered Content:**

  - Scrapy does not execute JavaScript. If the page content is loaded via JavaScript, consider using tools like **Scrapy Splash** or **Selenium** with Scrapy.

- **Improving Text Extraction:**

  - For better extraction of the main text, consider using libraries like `readability-lxml`.

In [ ]:
from readability import Document

def extract_main_text(self, response):
    doc = Document(response.text)
    main_html = doc.summary()
    main_soup = scrapy.Selector(text=main_html)
    text = main_soup.xpath('//text()').getall()
    text = [t.strip() for t in text if t.strip()]
    text_content = '\n'.join(text)
    return text_content

  - **Note:** You'll need to install `readability-lxml`:

    ```bash
    pip install readability-lxml
    ```

- **Single Page Crawl:**

  - The spider as written only crawls the single page specified in `start_urls`. If you want to follow links and crawl multiple pages, you need to adjust the `parse` method to yield new requests.

In [ ]:
for href in link_urls:
    full_url = urljoin(response.url, href)
    yield scrapy.Request(full_url, callback=self.parse)

  - Be cautious with recursive crawling to avoid crawling the entire website unintentionally.

- **Avoid Duplicates:**

  - Ensure that you're not collecting duplicate URLs by checking if a URL is already in your list before adding it.

- **Data Cleaning:**

  - You may need to clean the extracted data further, depending on your requirements.

### Important Considerations:

- **Ethical Scraping:**

  - Always scrape responsibly by following the website's `robots.txt` rules and terms of service.
  - Use a meaningful `User-Agent` string to identify your scraper.

- **Robustness:**

  - Add error handling to manage network issues or unexpected HTML structures.

- **Testing:**

  - Test your scraper on a small scale before running it on larger sites.

**Disclaimer:**

- **Use Responsibly:** Ensure that you use this code responsibly and ethically, complying with all applicable laws and website policies.

---

**Feel free to ask if you need further assistance or modifications to the code!**

## web browser automation

In [ ]:
#!pip install pytest-playwright
#!playwright install
#!pip install pytest-playwright playwright -U

#!pip install playwright
#!playwright install

In [ ]:
# screenshot.py

from playwright.sync_api import sync_playwright

def take_full_page_screenshot(url, output_path):
    with sync_playwright() as p:
        # Launch the browser (Chromium in headless mode)
        browser = p.chromium.launch()
        page = browser.new_page()
        # Navigate to the specified URL
        page.goto(url)
        # Wait until the page is fully loaded
        page.wait_for_load_state('networkidle')
        # Take a full-page screenshot
        page.screenshot(path=output_path, full_page=True)
        # Close the browser
        browser.close()

# Usage example
if __name__ == "__main__":
    url = 'https://www.example.com'  # Replace with your target URL
    output_path = 'full_screenshot.png'  # Output file path
    take_full_page_screenshot(url, output_path)
    print(f"Screenshot saved to {output_path}")

---

### **Explanation**

#### **Imports and Setup**

- **Import `sync_playwright`**: We import `sync_playwright` from `playwright.sync_api` to use Playwright in synchronous mode.


In [ ]:
from playwright.sync_api import sync_playwright

#### **Define the Function**

- **Function `take_full_page_screenshot`**: This function takes a URL and an output file path as arguments and captures a full-page screenshot of the specified URL.


In [ ]:
def take_full_page_screenshot(url, output_path):

#### **Start Playwright and Launch the Browser**

- **Use `sync_playwright()` Context Manager**: This ensures that Playwright starts and stops correctly.


In [ ]:
with sync_playwright() as p:

- **Launch the Browser**: We use Chromium in headless mode by default. You can choose `'chromium'`, `'firefox'`, or `'webkit'`.


In [ ]:
browser = p.chromium.launch()

- **Create a New Page**: Open a new browser page (tab).


In [ ]:
page = browser.new_page()

#### **Navigate to the URL**

- **Go to the Specified URL**: Navigate the browser to the target URL.


In [ ]:
page.goto(url)

- **Wait for the Page to Load**: Ensure the page is fully loaded before taking the screenshot.


In [ ]:
page.wait_for_load_state('networkidle')

#### **Take the Full-Page Screenshot**

- **Capture the Screenshot**: Use the `screenshot` method with `full_page=True` to capture the entire page.


In [ ]:
page.screenshot(path=output_path, full_page=True)

#### **Close the Browser**

- **Close the Browser**: Clean up by closing the browser instance.


In [ ]:
browser.close()

#### **Usage Example**

- **Provide an Example Usage**: Demonstrate how to call the function with a specific URL and output path.


In [ ]:
if __name__ == "__main__":
    url = 'https://www.example.com'  # Replace with your target URL
    output_path = 'full_screenshot.png'  # Output file path
    take_full_page_screenshot(url, output_path)
    print(f"Screenshot saved to {output_path}")

---

### **Step-by-Step Guide**

#### **1. Install Playwright**

First, you need to install Playwright and its dependencies.

```bash
pip install playwright
```

Then, install the browser binaries:

```bash
playwright install
```

This command installs Chromium, Firefox, and WebKit browsers needed by Playwright.

#### **2. Save the Script**

Save the code above in a file named `screenshot.py`.

#### **3. Replace the URL**

In the script, replace `'https://www.example.com'` with the URL of the page you want to capture.

In [ ]:
url = 'https://www.your-target-website.com'

#### **4. Run the Script**

Execute the script from the command line:

```bash
python screenshot.py
```

#### **5. Check the Output**

After running the script, you should find a file named `full_screenshot.png` in the same directory. This image is the full-page screenshot of the specified URL.

---

### **Additional Options**

#### **Specify Browser Options**

You can customize the browser launch options, such as running in headless or headful mode.

In [ ]:
browser = p.chromium.launch(headless=False)  # Set headless=False to see the browser

#### **Set Viewport Size**

If you need to set a specific viewport size:

In [ ]:
page = browser.new_page(viewport={'width': 1280, 'height': 720})

#### **Authentication and Headers**

If the page requires authentication or specific headers:

In [ ]:
page.goto(url, headers={'Authorization': 'Bearer YOUR_TOKEN'})

#### **Delay for Dynamic Content**

If the page loads content dynamically, you may need to wait before taking the screenshot.

In [ ]:
page.goto(url)
page.wait_for_timeout(5000)  # Wait for 5 seconds

Or wait for a specific element:

In [ ]:
page.goto(url)
page.wait_for_selector('#content')  # Wait for an element with id 'content'

---

### **Notes and Best Practices**

- **Error Handling**: You can add try-except blocks to handle exceptions.

In [ ]:
try:
    # Your code
except Exception as e:
    print(f"An error occurred: {e}")

- **Headless vs. Headful**: Running in headless mode (`headless=True`) means the browser runs in the background without a GUI. Setting `headless=False` allows you to see the browser actions.

- **Resource Cleanup**: Using the context manager ensures resources are properly cleaned up after execution.

- **Playwright Documentation**: For more advanced usage, refer to the official Playwright Python documentation: [Playwright for Python](https://playwright.dev/python/docs/intro)

---

### **Example with Advanced Features**

In [ ]:
from playwright.sync_api import sync_playwright

def take_full_page_screenshot(url, output_path):
    with sync_playwright() as p:
        # Launch the browser in headless mode
        browser = p.chromium.launch(headless=True)
        # Create a new browser context with a specific viewport size
        context = browser.new_context(
            viewport={'width': 1920, 'height': 1080}
        )
        page = context.new_page()
        # Navigate to the URL
        page.goto(url)
        # Wait for network to be idle
        page.wait_for_load_state('networkidle')
        # Wait for any animations to finish
        page.wait_for_timeout(1000)  # Wait for 1 second
        # Hide any unwanted elements (e.g., cookie banners)
        page.evaluate("() => { const banners = document.querySelectorAll('.cookie-banner'); banners.forEach(b => b.style.display = 'none'); }")
        # Take a full-page screenshot
        page.screenshot(path=output_path, full_page=True)
        # Close the browser
        browser.close()

# Usage example
if __name__ == "__main__":
    url = 'https://www.example.com'  # Replace with your target URL
    output_path = 'full_screenshot.png'  # Output file path
    take_full_page_screenshot(url, output_path)
    print(f"Screenshot saved to {output_path}")

**Explanation of Additional Features:**

- **Viewport Size**: Setting the viewport to a larger size can capture more content.

- **Waiting for Animations**: Adding a timeout after page load can ensure that any animations or dynamic content have completed.

- **Hiding Elements**: Use `evaluate` to run JavaScript in the page context to hide unwanted elements like pop-ups or banners.

---

### **Common Issues and Troubleshooting**

- **Timeout Errors**: If the page takes too long to load, you might encounter timeout errors. Increase the timeout using:

In [ ]:
page.goto(url, timeout=60000)  # Wait up to 60 seconds

- **SSL Certificate Errors**: For pages with SSL issues, you can ignore HTTPS errors:

In [ ]:
browser = p.chromium.launch(args=['--ignore-certificate-errors'])

- **Authentication Required**: For pages behind authentication, provide credentials:

In [ ]:
context = browser.new_context(
    http_credentials={'username': 'user', 'password': 'pass'}
)

---

### **Conclusion**

This script provides a simple way to use Playwright for Python to visit a URL and take a full-page screenshot, saving it as a PNG file. Playwright is a powerful tool for browser automation and supports advanced features that can be leveraged for more complex tasks.

---

**Remember to replace `'https://www.example.com'` with the actual URL you want to capture and ensure you comply with the website's terms of service and robots.txt policies when automating interactions.**
